In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('Cleaned_Data')
df.head()

,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,State,Age,Important_Customer
0,Female,93,Executive Secretary,Health,Mass Customer,Yes,11.0,10,NSW,Senior_Citizen,1
1,Male,81,Administrative Officer,Financial Services,Mass Customer,Yes,16.0,10,NSW,Middle_Aged,0
2,Male,33,NaN,IT,Mass Customer,No,7.0,9,QLD,Senior_Citizen,0
3,Female,56,Senior Editor,NaN,Affluent Customer,Yes,8.0,4,NSW,Middle_Aged,0
4,Male,35,NaN,Retail,High Net Worth,Yes,13.0,9,VIC,Middle_Aged,0


In [3]:
df.past_3_years_bike_related_purchases.min(),df.past_3_years_bike_related_purchases.max()

(0, 99)

In [4]:
def past_bin(r):
    if r<10:
        return 0
    elif r<20:
        return 1
    elif r<30:
        return 2
    elif r<40:
        return 3
    elif r<50:
        return 4
    elif r<60:
        return 5
    elif r<70:
        return 6
    elif r<80:
        return 7
    elif r<90:
        return 8
    else:
        return 9

In [5]:
df['past_3_years_purchase']=df.past_3_years_bike_related_purchases.apply(past_bin)
df.head(2)

,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,State,Age,Important_Customer,past_3_years_purchase
0,Female,93,Executive Secretary,Health,Mass Customer,Yes,11.0,10,NSW,Senior_Citizen,1,9
1,Male,81,Administrative Officer,Financial Services,Mass Customer,Yes,16.0,10,NSW,Middle_Aged,0,8


In [6]:
df=df.drop('past_3_years_bike_related_purchases',1)
df.head(1)

,gender,job_title,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,State,Age,Important_Customer,past_3_years_purchase
0,Female,Executive Secretary,Health,Mass Customer,Yes,11.0,10,NSW,Senior_Citizen,1,9


In [7]:
df.tenure.min(),df.tenure.max()

(1.0, 22.0)

In [8]:
def ten_bin(r):
    if r<5:
        return 0
    elif r<10:
        return 1
    elif r<15:
        return 2
    else:
        return 3

In [9]:
df['Tenure']=df.tenure.apply(ten_bin)
df.head(2)

,gender,job_title,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,State,Age,Important_Customer,past_3_years_purchase,Tenure
0,Female,Executive Secretary,Health,Mass Customer,Yes,11.0,10,NSW,Senior_Citizen,1,9,2
1,Male,Administrative Officer,Financial Services,Mass Customer,Yes,16.0,10,NSW,Middle_Aged,0,8,3


In [10]:
df=df.drop('tenure',1)
df.head(1)

,gender,job_title,job_industry_category,wealth_segment,owns_car,property_valuation,State,Age,Important_Customer,past_3_years_purchase,Tenure
0,Female,Executive Secretary,Health,Mass Customer,Yes,10,NSW,Senior_Citizen,1,9,2


In [11]:
X=df.drop('Important_Customer',1)
y=df['Important_Customer']

In [12]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(X,y, random_state=10,test_size=0.2)

In [13]:
from xverse.transformer import WOE
clf=WOE()

In [14]:
clf.fit(xtrain,ytrain)
xtrain_woe=clf.transform(xtrain)
xtest_woe=clf.transform(xtest)

C:\Users\Shakti\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [15]:
xtrain_woe.State.value_counts()

-0.033104    1487
 0.041962     708
 0.036341     589
-0.386103       5
Name: State, dtype: int64

In [16]:
from sklearn.metrics import accuracy_score,f1_score

# Lets define a function for dataframe

df_me=pd.DataFrame(columns=['Model','Acc','f1score'])
def update(model,model_name):
    model.fit(xtrain_woe,ytrain)
    pred=model.predict(xtest_woe)
    
    global df_me
    df_me=df_me.append({'Model':model_name,'Acc':accuracy_score(ytest,pred),'f1score':f1_score(ytest,pred)},ignore_index=True)
    return df_me

from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV,validation_curve

In [17]:
rf_best=RandomForestClassifier(n_estimators=70,criterion='gini',max_depth=6,random_state=10)

update(rf_best,'RF_best')

,Model,Acc,f1score
0,RF_best,0.515759,0.498516


In [18]:
ad=AdaBoostClassifier()
param=[{'learning_rate':np.arange(0.02,0.08,0.01),'n_estimators': np.arange(20,60,10)}]
ad_grid=GridSearchCV(estimator=ad,param_grid=param)

ad_grid.fit(xtrain_woe,ytrain)

ad_grid.best_params_

{'learning_rate': 0.06, 'n_estimators': 40}

In [19]:
ad_best_2=AdaBoostClassifier(learning_rate=0.06,n_estimators=40,random_state=10)

update(ad_best_2,'AdaB_best')

,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093


In [20]:
param=[{'learning_rate':np.arange(0.02,0.08,0.01),'n_estimators': np.arange(20,60,10),'loss':['deviance','exponential'],
       'max_depth':np.arange(1,3,1)}]

gb_grid=GridSearchCV(estimator=GradientBoostingClassifier(),param_grid=param)

gb_grid.fit(xtrain_woe,ytrain)

gb_grid.best_params_

{'learning_rate': 0.049999999999999996,
 'loss': 'deviance',
 'max_depth': 2,
 'n_estimators': 30}

In [21]:
gb_best=GradientBoostingClassifier(learning_rate=0.05,loss='deviance',max_depth=2,n_estimators=30,random_state=10)

update(gb_best,'GB_Best')

,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669


In [22]:
from xgboost import XGBClassifier

xgb=XGBClassifier()

update(xgb,'XGBOOST')

C:\Users\Shakti\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:49:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286


In [24]:
import sklearn


In [25]:
from sklearn.neighbors import KNeighborsClassifier
KNN=KNeighborsClassifier()

In [26]:
param=[{'n_neighbors':np.arange(4,20,1),'p':np.arange(2,5,1)}]


KN_grid=GridSearchCV(estimator=KNN,param_grid=param)
KN_grid.fit(xtrain_woe,ytrain)

KN_grid.best_params_

{'n_neighbors': 18, 'p': 2}

In [27]:
KNN_best=KNeighborsClassifier(n_neighbors=18)

update(KNN_best,'KNN_best')

,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511


In [28]:
from sklearn.naive_bayes import GaussianNB
gb=GaussianNB()

In [29]:
update(gb,'GausianNB')

,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
LR=LogisticRegression()

update(LR,'LOG_REG')

,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582


In [31]:
from sklearn.ensemble import StackingClassifier

# consider the various algorithms as base learners
base_learners = [('rf_model', RandomForestClassifier(n_estimators=70,criterion='gini',max_depth=6,random_state=10)),
                 ('KNN_model', KNeighborsClassifier(n_neighbors = 18, metric = 'euclidean')),
                 ('NB_model', GaussianNB()),
                ('GradientBoost_model',GradientBoostingClassifier(learning_rate=0.05,loss='deviance',max_depth=2,n_estimators=30,random_state=10)),
                ('XGB_model',XGBClassifier())]

# initialize stacking classifier 
# pass the base learners to the parameter, 'estimators'
# pass the Naive Bayes model as the 'final_estimator'/ meta model
stack_model = StackingClassifier(estimators = base_learners, final_estimator = GaussianNB())

# fit the model on train dataset
update(stack_model,'Stack_Model')

C:\Users\Shakti\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:22:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Shakti\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:22:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Shakti\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\Shakti\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:22:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Shakti\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:22:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879


In [32]:
# KNN gives the best accuracy.


In [1]:
#Lets run KNN again for better result

In [33]:
KNN_best=KNeighborsClassifier()

#We got best n_neigbors=18,p=2 for minkowski distance

In [41]:
import sklearn
import sklearn.metrics._dist_metrics as dm

In [50]:
dist_mat =sorted(sklearn.neighbors.VALID_METRICS['brute'])   

In [76]:
dist_mat

['braycurtis',
 'canberra',
 'chebyshev',
 'cityblock',
 'correlation',
 'cosine',
 'cosine',
 'dice',
 'euclidean',
 'hamming',
 'haversine',
 'jaccard',
 'kulsinski',
 'l1',
 'l2',
 'mahalanobis',
 'manhattan',
 'matching',
 'minkowski',
 'nan_euclidean',
 'precomputed',
 'rogerstanimoto',
 'russellrao',
 'seuclidean',
 'sokalmichener',
 'sokalsneath',
 'sqeuclidean',
 'wminkowski',
 'yule']

In [ ]:
new_dist=['braycurtis',
'euclidean',
'l1',
'minkowski',
'rogerstanimoto',
'sqeuclidean']

In [111]:
param=[{'n_neighbors':range(18,19),'weights':['uniform', 'distance'],
        'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
       'metric':['braycurtis']}]

kn_grid=GridSearchCV(estimator=KNN_best,param_grid=param)

In [112]:
kn_grid.fit(xtrain_woe,ytrain)

C:\Users\Shakti\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Shakti\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Shakti\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py", line 198, in fit
    return self._fit(X, y)
  File "C:\Users\Shakti\anaconda3\lib\site-packages\sklearn\neighbors\_base.py", line 437, in _fit
    self._check_algorithm_metric()
  File "C:\Users\Shakti\anaconda3\lib\s

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid=[{'algorithm': ['auto', 'ball_tree', 'kd_tree',
                                        'brute'],
                          'metric': ['braycurtis'],
                          'n_neighbors': range(18, 19),
                          'weights': ['uniform', 'distance']}])

In [113]:
kn_grid.best_params_

{'algorithm': 'brute',
 'metric': 'braycurtis',
 'n_neighbors': 18,
 'weights': 'distance'}

In [116]:
KN_braycurtis=KNeighborsClassifier(n_neighbors=18,algorithm='brute',metric='braycurtis',weights='distance')
update(KN_braycurtis,'KNN_Braycurtis')

,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879
8,KNN_Braycurtis,0.492837,0.488439


In [66]:
KN=KNeighborsClassifier()


['correlation', 'cosine', 'cosine', 'dice']

In [117]:
param=[{'n_neighbors':range(18,19),'weights':['uniform', 'distance'],
        'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],'p':np.arange(0.5,1.5,0.1)}]

kn_grid=GridSearchCV(estimator=KNN_best,param_grid=param)
kn_grid.fit(xtrain_woe,ytrain)
kn_grid.best_params_

C:\Users\Shakti\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
240 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Shakti\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Shakti\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py", line 198, in fit
    return self._fit(X, y)
  File "C:\Users\Shakti\anaconda3\lib\site-packages\sklearn\neighbors\_base.py", line 437, in _fit
    self._check_algorithm_metric()
  File "C:\Users\Shakti\anaconda3\li

{'algorithm': 'ball_tree',
 'n_neighbors': 18,
 'p': 1.2999999999999998,
 'weights': 'uniform'}

In [118]:
KN_MIN=KNeighborsClassifier(n_neighbors=18,algorithm='ball_tree',p=1.3,weights='uniform')
update(KN_MIN,'knn_mink')

,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879
8,KNN_Braycurtis,0.492837,0.488439
9,knn_mink,0.531519,0.508271


In [122]:
param=[{'n_neighbors':range(18,19),
        'algorithm' : [ 'ball_tree'],'p':np.arange(1.25,1.35,0.01)}]

kn_grid=GridSearchCV(estimator=KNN_best,param_grid=param)
kn_grid.fit(xtrain_woe,ytrain)
kn_grid.best_params_

{'algorithm': 'ball_tree', 'n_neighbors': 18, 'p': 1.3}

In [123]:
KNN_=KNeighborsClassifier(n_neighbors=18,algorithm='ball_tree',p=1.3)
update(KNN_,'The_Best_KNN')

,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879
8,KNN_Braycurtis,0.492837,0.488439
9,knn_mink,0.531519,0.508271


In [ ]:
# Not much change than previous which means model has saturated.

In [126]:
# Lets check for Logit model again
import statsmodels.api as sm

logreg=sm.Logit(ytrain,xtrain_woe).fit()

Optimization terminated successfully.
         Current function value: 0.657784
         Iterations 5


In [127]:
y_pred_prob = logreg.predict(xtest_woe)

from sklearn.metrics import roc_curve

fpr,tpr,th=roc_curve(ytest,y_pred_prob)

th_df=pd.DataFrame()
th_df['TH']=th
th_df['FPR']=fpr
th_df['TPR']=tpr

th_df['diff']=th_df.TPR-th_df.FPR


In [128]:
th_df.sort_values('diff',ascending=False)

,TH,FPR,TPR,diff
98,0.598188,0.224852,0.277778,0.052926
96,0.599113,0.221893,0.272222,0.050329
62,0.643298,0.139053,0.188889,0.049836
100,0.596272,0.230769,0.280556,0.049786
64,0.642188,0.142012,0.191667,0.049655
...,...,...,...,...
360,0.281184,0.967456,0.952778,-0.014678
362,0.278535,0.970414,0.955556,-0.014859
255,0.435951,0.710059,0.694444,-0.015615
370,0.255977,0.988166,0.972222,-0.015943


In [132]:
y_pred=[0 if x<0.598188 else 1 for x in y_pred_prob]

from sklearn.metrics import classification_report,accuracy_score,f1_score
report=classification_report(ytest,y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.50      0.78      0.61       338
           1       0.57      0.28      0.37       360

    accuracy                           0.52       698
   macro avg       0.53      0.53      0.49       698
weighted avg       0.53      0.52      0.49       698



In [133]:
accuracy_score(ytest,y_pred)

0.5171919770773639

In [134]:
f1_score(ytest,y_pred)  # Very low f1 score.

0.37009345794392523

In [135]:
base_learners = [
                 ('KNN_model_1', KNeighborsClassifier(n_neighbors=18,algorithm='ball_tree',p=1.3)),
                 ('KNN_model_2', KNeighborsClassifier(n_neighbors=18))]

# initialize stacking classifier 
# pass the base learners to the parameter, 'estimators'
# pass the Naive Bayes model as the 'final_estimator'/ meta model
stack_model = StackingClassifier(estimators = base_learners, final_estimator = GaussianNB())

# fit the model on train dataset
update(stack_model,'Stack_Model_GNB')



,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879
8,KNN_Braycurtis,0.492837,0.488439
9,knn_mink,0.531519,0.508271


In [136]:
from sklearn.tree import DecisionTreeClassifier

In [137]:
base_learners = [
                 ('KNN_model_1', KNeighborsClassifier(n_neighbors=18,algorithm='ball_tree',p=1.3)),
                 ('KNN_model_2', KNeighborsClassifier(n_neighbors=18))]

# initialize stacking classifier 
# pass the base learners to the parameter, 'estimators'
# pass the Naive Bayes model as the 'final_estimator'/ meta model
stack_model = StackingClassifier(estimators = base_learners, final_estimator = DecisionTreeClassifier())

# fit the model on train dataset
update(stack_model,'Stack_Model_DT')


,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879
8,KNN_Braycurtis,0.492837,0.488439
9,knn_mink,0.531519,0.508271


In [138]:
base_learners = [
                 ('KNN_model_1', KNeighborsClassifier(n_neighbors=18,algorithm='ball_tree',p=1.3)),
                 ('KNN_model_2', KNeighborsClassifier(n_neighbors=18))]

# initialize stacking classifier 
# pass the base learners to the parameter, 'estimators'
# pass the Naive Bayes model as the 'final_estimator'/ meta model
stack_model = StackingClassifier(estimators = base_learners, final_estimator = RandomForestClassifier())

# fit the model on train dataset
update(stack_model,'Stack_Model_RF')


,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879
8,KNN_Braycurtis,0.492837,0.488439
9,knn_mink,0.531519,0.508271


In [139]:
base_learners = [
                 ('KNN_model_1', KNeighborsClassifier(n_neighbors=18,algorithm='ball_tree',p=1.3)),
                 ('KNN_model_2', KNeighborsClassifier(n_neighbors=18))]

# initialize stacking classifier 
# pass the base learners to the parameter, 'estimators'
# pass the Naive Bayes model as the 'final_estimator'/ meta model
stack_model = StackingClassifier(estimators = base_learners, final_estimator = KNeighborsClassifier())

# fit the model on train dataset
update(stack_model,'Stack_Model_KNN')


,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879
8,KNN_Braycurtis,0.492837,0.488439
9,knn_mink,0.531519,0.508271


In [140]:
base_learners = [
                 ('KNN_model_1', KNeighborsClassifier(n_neighbors=18,algorithm='ball_tree',p=1.3)),
                 ('KNN_model_2', KNeighborsClassifier(n_neighbors=18))]

# initialize stacking classifier 
# pass the base learners to the parameter, 'estimators'
# pass the Naive Bayes model as the 'final_estimator'/ meta model
stack_model = StackingClassifier(estimators = base_learners, final_estimator = GradientBoostingClassifier())

# fit the model on train dataset
update(stack_model,'Stack_Model_GBC')


,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879
8,KNN_Braycurtis,0.492837,0.488439
9,knn_mink,0.531519,0.508271


In [141]:
base_learners = [
                 ('KNN_model_1', KNeighborsClassifier(n_neighbors=18,algorithm='ball_tree',p=1.3)),
                 ('KNN_model_2', KNeighborsClassifier(n_neighbors=18))]

# initialize stacking classifier 
# pass the base learners to the parameter, 'estimators'
# pass the Naive Bayes model as the 'final_estimator'/ meta model
stack_model = StackingClassifier(estimators = base_learners, final_estimator = XGBClassifier())

# fit the model on train dataset
update(stack_model,'Stack_Model_XGB')


C:\Users\Shakti\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:56:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879
8,KNN_Braycurtis,0.492837,0.488439
9,knn_mink,0.531519,0.508271


In [142]:
base_learners = [
                 ('KNN_model_1', KNeighborsClassifier(n_neighbors=18,algorithm='ball_tree',p=1.3)),
                 ('KNN_model_2', KNeighborsClassifier(n_neighbors=18))]

# initialize stacking classifier 
# pass the base learners to the parameter, 'estimators'
# pass the Naive Bayes model as the 'final_estimator'/ meta model
stack_model = StackingClassifier(estimators = base_learners, final_estimator = AdaBoostClassifier())

# fit the model on train dataset
update(stack_model,'Stack_Model_ABC')


,Model,Acc,f1score
0,RF_best,0.515759,0.498516
1,AdaB_best,0.504298,0.497093
2,GB_Best,0.504298,0.492669
3,XGBOOST,0.512894,0.514286
4,KNN_best,0.527221,0.501511
5,GausianNB,0.487106,0.473529
6,LOG_REG,0.510029,0.492582
7,Stack_Model,0.515759,0.487879
8,KNN_Braycurtis,0.492837,0.488439
9,knn_mink,0.531519,0.508271


In [ ]:
#Stack model XGB gives a balance between sccuracy and f1score.

In [ ]:
base_learners = [('KNN_model_1', KNeighborsClassifier(n_neighbors=18,algorithm='ball_tree',p=1.3)),
                 ('KNN_model_2', KNeighborsClassifier(n_neighbors=18))]

stack_model = StackingClassifier(estimators = base_learners, final_estimator = XGBClassifier())



